In [4]:
# Trace evaluation domain and FRI evaluation domain
# G = trace evaluation domain
# L = FRI evaluation domain
# G and L are disjoint
# |G| < |L|
# |G| is a power of 2 (for fast interpolation of polynomial?)
# |L| is a power of 2 (for fast evaluation of polynomial)
# F[P] = Prime field of order P, P is prime
# F[P, *] = Multiplicative subgroup of F[P] = {1, 2, 3, ..., P - 1}
# |F[P, *]| = P - 1
# G and L are subgroups of F[P, *] -> |G| and |L| divides |F[P, *]| = P - 1
from utils import is_prime, is_pow2
from field import find_generator, check_cyclic

P = 3 * 2**30 + 1
assert is_prime(P)

### G = trace evaluation domain ###
# Find G such that |G| = 1024
# T must divide P - 1, 1024 = 2**10
T = 1024
assert is_pow2(T)
# Generator of F[P, *] - multiplicative group of prime field F[P]
g = find_generator(P)
print("Generator of F[P, *] =", g)

# Generator of G
# When k divides |F[P, *]|, g^k generates a group of size |F[P, *]| / k
# |F[P, *]| / k = |G| -> k = (P - 1) / |G|
G_gen = pow(g, (P - 1) // T, P)
print("Generator of G =", G_gen)
G = [pow(G_gen, i, P) for i in range(T)]

check_cyclic(G_gen, G, P, T)

### L = FRI evaluation domain ###
# |L| = EXP_FACTOR * |G|
# L and G are disjoint
EXP_FACTOR = 8
# |F[P, *]| / k = EXP_FACTOR * |G| -> k = (P - 1) / (EXP_FACTOR * |G|)
H_gen = pow(g, (P - 1) // (EXP_FACTOR * T), P)
print("Generator of L =", H_gen)
H = [pow(H_gen, i, P) for i in range(EXP_FACTOR * T)]

check_cyclic(H_gen, H, P, EXP_FACTOR * T)

# Coset shifted by g = {g* H}
L = [(g * h) % P for h in H]
assert len(set(L)) == len(H), f'|L| = {len(set(L))}'
assert (set(L) & set(G)) == set(), f'L and G are not disjoint {set(L) & set(G)}'

Generator of F[P, *] = 5
Generator of G = 1855261384
Generator of L = 1734477367


In [ ]:
# A Walk-Through of a Simple zk-STARK Proof

# N = 4
# A = [1, 0, 1, 1]
# g = 4 mod 17
# f(g[i]) = A[i] for i < N
# A[i] = 0 or 1 -> A[i](A[i] - 1) = 0 for i < N -> f(g[i])^2 - f(g[i]) = 0
# c(x) = f(x)^2 - f(x)
# u(x) = ((x-g[0])(x - g[1])...(x - g[N - 1])) = x^N - 1
# p(x) = c(x) / u(x)
# p(x) is polynomial of degree <= deg(c) - N <-> c(x) = 0 for x = g[i] <-> All A[i] = 0 or 1

# Query
# Verifier checks p(x)u(x) = c(x) (query x in L - G, query x in G breaks zero knowledge)
# 1. Verifier queries for z in L - G
# 2. Prover sends p(z) and f(z) and Merkle proofs of p(z) and f(z)
# 3. Verifier checks p(z)u(z) = c(z) and Merkle proofs

# num queries for high probability of fraud detection
# poly degree = d = 5
# evaluation domain L = 100
# num of dishonest evaluation = |L| - d = 95
# probability of getting caught after k queries = 1 - probability of not getting caught after k queries ~ 1 - (d / |L|)^k

# TODO - cheat if p is not a low degree
# TODO - example of false trace?

# Apply FRI to p

In [3]:
import numpy as np
from fft import fft, ifft, eval_poly
from field import F
import polynomial
from polynomial import Polynomial, X
import merkle
from utils import min_pow2_gt

# Trace
T = [1, 0, 1, 1]
M = len(T)

# F[P] = prime field
# P = 17
# g = 4
# w = 3
# P = 97
# g = 5
# w = 8
P = 17
# Generator
g = 4
# Trace evaluation domain
G = [pow(g, i, P) for i in range(M)]
print("G", G)

def wrap(x: int) -> F:
    return F(x, P)

# Trace polynomial f(G[i]) = A[i]
f = polynomial.interp(G, T, wrap)
print("f(G)", f(G))
assert f(G) == T

# G < L <= F[P] - {0}
# |L| = N
N = 16
assert M < N

# Primitive Nth root
w = 3
assert pow(w, N, P) == 1
assert pow(w, N // 2, P) == (-1 % P)

# TODO: Shift L so that all elements of G are not in L (hide f(G))?
# TODO: shift is provided by prover or verifier?
# FRI evaluation domain
L = [pow(w, i, P) for i in range(N)]
print("L", L)
print("L & G", set(L) & set(G))
assert len(set(L)) == N

# Constraint polynomial c(x) = f(x)^2 - f(x) = A[i](A[i] - 1) = 0 for G[i]
c = f*f - f

# Composition polynomial (RISCZERO calls in validity polynomial) 
# z(x) = (x - g^0)(x - g^1)...(x - g^(M-1)) = x^M - 1
# q(x) = c(x) / z(x)

# Why divide c(x) by z(x)? 
# Check c(x) -> need to check c(g^i) = 0 for 0 <= i < M
# c(g^i) = 0 for 0 <= i < M <-> g^i is a root of c <-> z divides c (c(x) / z(x) = 0 mod P)
# Dividing by z, we only do 1 check instead of M checks

# NOTE: in production several constraints are combined pseudo randomly with a1, a2, ...
# a1 * q1(x) + a2 * q2(x) + ... for constraints q1(x), q2(x), ...

# z(x) = x^M - 1
z = X(M) - 1
q = c / z

# Highest degree of all constraint polynomials claimed by the prover
MAX_LOW_DEGREE = 2

print("q =", q)
print("Degree of q =", q.degree())
assert q.degree() == MAX_LOW_DEGREE

# Check q(G[i]) != 0
assert all(y != 0 for y in q(G))

# Degree adjustment
# Let max_deg = highest degree of all C[j] where C[j] are constraint polynomials
# Let D = 2**k where k is smallest such that D > max_deg
# Adjust degree of C[j] to D - 1
# Given C[j] with degree of C[j] = D[j]
# Degree adjusted polynomial = C[j](x) * (A[j] * x^(D - D[j] - 1) + B[j])
# where A[j] and B[j] are random values provided by the verifier
deg_adj = min_pow2_gt(MAX_LOW_DEGREE)
assert deg_adj > MAX_LOW_DEGREE
# TODO: get challenges from verifier
a = 1
b = 2
adj = a * X(deg_adj - MAX_LOW_DEGREE - 1) + b
q = q * adj
print("Adjusted degree of q = ", q.degree())

# Prover sends Merkle root of q(x) and f(x)
hashes_f = [merkle.hash_leaf(str(y)) for y in f(L)]
hashes_q = [merkle.hash_leaf(str(y)) for y in q(L)]
root_f = merkle.commit(hashes_f)
root_q = merkle.commit(hashes_q)

print("Merkle root of q =", root_q) 

# Verifier checks q(x)z(x) = c(x) for random x, x not in G
NUM_QUERIES = 5
assert NUM_QUERIES <= N - M
# Q = Random selections without replacement from L - G
Q = [(i, v) for i, v in enumerate(L) if v not in G]
Q = [Q[i] for i in np.random.choice(range(len(Q)), size = NUM_QUERIES, replace = False)]

for (i, x) in Q:
    # Prover sends f(x), q(x) and Merkle proofs
    fx = f(x)
    qx = q(x)
    proof_f = merkle.open(hashes_f, i)
    proof_q = merkle.open(hashes_q, i)

    # TODO: is this correct?
    # q(x) = c(x) / z(x)
    # q_adj(x) = q(x) * adj(x)
    # q_adj(x) * z(x) = c(x) * adj(x)
    
    # Verifier checks q_adj(x)z(x) = c(x) * adj(x) and Merkle proofs of f and q_adj
    # Verifier can compute adj(x)
    cx = fx * fx - fx
    zx = z(x)
    adjx = adj(x)
    assert qx * zx == cx * adjx

    assert merkle.verify(proof_f, root_f, hashes_f[i], i)
    assert merkle.verify(proof_q, root_q, hashes_q[i], i)

G [1, 4, 16, 13]
f(G) [1, 0, 1, 1]
L [1, 3, 9, 10, 13, 5, 15, 11, 16, 14, 8, 7, 4, 12, 2, 6]
L & G {16, 1, 4, 13}
q = [14, 8, 1]
Degree of q = 2
Adjusted degree of q =  3
Merkle root of q = 7594c30688c3de46d38fc5ad781c227fcf09db22b81c3c0bb0642b9dd6b3efbc


In [2]:
# Low degree testing - Verifier checks q(x) is low degree

import numpy as np
import fri
from field import F
import polynomial
from polynomial import Polynomial
import merkle
import iop
from utils import is_pow2, is_prime

# TODO: FRI domain L = disjoint set from trace domain G
# TODO: pair f(x) and f(-x) in merkle tree?

# FRI domain L
# message length = M -> poly degree < M -> RS code length = N = |L| size of evaluation domain
# N is a power of 2 = 2**k for some k
# L = [1, w, w^2, ..., w^(N - 1)], w is primitive Nth root of unity
# prime field F_p, |F_p| > |L|, N divides p - 1 (needed for finding primitive root of unity)

# Use to test high degree
# q = Polynomial([1, 2, 3, 4, 5], lambda x: F(x, P))
assert q.degree() == M - 1

# Expansion factor from message length M to RS code length N
# exp_factor * M = N
EXP_FACTOR = 4
assert M * EXP_FACTOR == N

assert is_prime(P), f'{P} is not prime'
assert is_pow2(N), f'{N} is not a power of 2'

prover = fri.Prover(
    N = N,
    P = P,
    w = w,
    shift = 1,
    exp_factor = EXP_FACTOR,
)

verifier = fri.Verifier(
    N = N,
    P = P,
    w = w,
    shift = 1,
    exp_factor = EXP_FACTOR
)

### Communication channels ###
# Prover to verifier channel
p_chan = iop.Channel(iop.Verifier(verifier))
# Verifier to prover channel
v_chan = iop.Channel(iop.Prover(prover))

### Commit ###
prover.commit(fri.eval_poly(q, L, P), p_chan)

print("Merkle root of q = ", prover.merkle_roots[0])
# assert prover.merkle_roots[0] == root_q
print("codewords", prover.codewords)

### Query ###
# Random indexes without replacement
NUM_QUERIES = 4
assert NUM_QUERIES <= N

# TODO - query q(x) for x not in G -> offset for q(L)?
Q = [int(i) for i in np.random.choice(range(N), size = NUM_QUERIES, replace = False) if L[i] not in G]
for i in Q:
    verifier.query(i, v_chan)

# TODO - attacks

Merkle root of q =  7594c30688c3de46d38fc5ad781c227fcf09db22b81c3c0bb0642b9dd6b3efbc
codewords [[1, 14, 1, 16, 4, 9, 0, 9, 7, 1, 9, 0, 15, 3, 0, 2], [6, 7, 16, 12, 10, 9, 0, 4], [8, 8, 8, 8]]


In [1]:
from fft import fft_rec, fft, ifft, eval_poly
P = 17
N = 16
g = 3

# Multiplicative group
gs = [pow(g, i, P) for i in range(P - 1)]
assert len(set(gs)) == P - 1

# N-th primitive root
w = pow(g, (P - 1)//N, P)
print("w", w)
# Check w is a N-th primitive root
assert pow(w, N, P) == 1
assert pow(w, N//2, P) == -1 % P

# FFT evaluation domain
ws = [pow(w, i, P) for i in range(N)]
# Example polynomial
f = [i + 1 for i in range(N)]
print("f", f)
print("ws", ws)

e = eval_poly(f, ws, P)
r = fft_rec(f, ws, P)
l = fft(f, ws, P)
i = ifft(l, ws, P)

print("e", e)
print("r", r)
print("l", l)
print("i", i)
assert r == e
assert l == e
assert i == f

# Evaluate a polynomial on a domain larger than the degree of the polynomial
f = [i + 1 for i in range(N//2)] + ([0] * (N//2))
print("f", f)

ys = fft(f, ws, P)
print("ys", ys)

cs = ifft(ys, ws, P)
print("cs", cs)

assert cs == f

w 3
f [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
ws [1, 3, 9, 10, 13, 5, 15, 11, 16, 14, 8, 7, 4, 12, 2, 6]
e [0, 8, 2, 15, 7, 4, 6, 5, 9, 13, 12, 14, 11, 3, 16, 10]
r [0, 8, 2, 15, 7, 4, 6, 5, 9, 13, 12, 14, 11, 3, 16, 10]
l [0, 8, 2, 15, 7, 4, 6, 5, 9, 13, 12, 14, 11, 3, 16, 10]
i [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
f [1, 2, 3, 4, 5, 6, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0]
ys [2, 5, 1, 9, 12, 13, 3, 5, 13, 0, 6, 11, 14, 8, 8, 8]
cs [1, 2, 3, 4, 5, 6, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0]


In [2]:
# N-th root of unity
# Any number w such that w^N = 1 mod P

# Primitive N-th root of unity
# w^N = 1 and w^k != 1 for 0 < k < N 

# Steps to find the N-th primitive root of unity mod P
# 0. Pick a prime P -> This defines the finite field F[P]
#    TODO: Why prime?
#    - nonzero element has a multiplicative inverse
# 1. Consider the multiplicative group F[P]x = {1,2,3,...,P−1} 
# 2. Find a generator g of F[P]x
# 3. Find N = largest power of 2 dividing P−1
#    The subgroup of order N is:
#      H = { g^((P−1)/N)^i  for i = 0 ... N−1 }
# 4. To find a primitive N-th root of unity:
#    Let w = g^((P−1)/N) (g^(P-1) = 1 (Fermat's little theorem))
#    Verify:
#      - w^N = 1
#      TODO: why this check suffices
#      - w^(N/2) = -1

from field import find_generator
from utils import max_log2

# 0. Pick a prime P
# Starkware prime
P = 2**251 + 17 * 2**192 + 1
# P = 17

# 2. Find a generator g
g = find_generator(P)
print("g = ", g)

# 3. Find N = largest power of 2 dividing P - 1
k = max_log2(P - 1)
N = 2**k
print("k =", k)
print("n =", N)

w = pow(g, ((P - 1) // N), P)

assert pow(w, N, P) == 1
assert pow(w, N // 2, P) == (-1 % P)

print(f"{N} th primitive root of unity =", w)

g =  3
k = 192
n = 6277101735386680763835789423207666416102355444464034512896
6277101735386680763835789423207666416102355444464034512896 th primitive root of unity = 145784604816374866144131285430889962727208297722245411306711449302875041684


Generator of F[P, *] = 5
Generator of G = 1855261384
Generator of L = 1734477367


1024